<a href="https://colab.research.google.com/github/psikrishna/shakespeare-shiptoasts/blob/master/shiptoasts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#importing keras etc for lstm
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
import keras.utils as ku 
#seeds
import tensorflow
tensorflow.random.set_seed(2)
from numpy.random import seed
seed(1)
import pandas as pd
import numpy as np
import string, os 
#i hate those warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
#dataset
curr_dir = '/content/sample_data/'
play_df = pd.read_csv('/content/sample_data/Shakespeare_data.csv')
all_lines = [h for h in play_df.PlayerLine]

In [8]:
#clean data
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 
corpus = [clean_text(x) for x in all_lines]
corpus[:10]

['act i',
 'scene i london the palace',
 'enter king henry lord john of lancaster the earl of westmoreland sir walter blunt and others',
 'so shaken as we are so wan with care',
 'find we a time for frighted peace to pant',
 'and breathe shortwinded accents of new broils',
 'to be commenced in strands afar remote',
 'no more the thirsty entrance of this soil',
 'shall daub her lips with her own childrens blood',
 'nor more shall trenching war channel her fields']

In [9]:
#tokenise
#tokens to help in skip-gram model
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    corpus = corpus[:7000]
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    #data->tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[523, 4],
 [142, 4],
 [142, 4, 339],
 [142, 4, 339, 1],
 [142, 4, 339, 1, 670],
 [53, 41],
 [53, 41, 84],
 [53, 41, 84, 29],
 [53, 41, 84, 29, 124],
 [53, 41, 84, 29, 124, 3]]

In [10]:
#padding
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
predictors.shape, label.shape

((45584, 33), (45584, 6543))

In [11]:
#design lstm
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(512))
    model.add(Dropout(0.4))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model
model = create_model(max_sequence_len, total_words)

In [24]:
model.fit(predictors, label, epochs=20, verbose=1)

Epoch 1/20
45584/45584 [==============================] - 94s 2ms/step - loss: 1.4549
Epoch 2/20
45584/45584 [==============================] - 94s 2ms/step - loss: 1.4657
Epoch 3/20
45584/45584 [==============================] - 95s 2ms/step - loss: 1.4623
Epoch 4/20
45584/45584 [==============================] - 92s 2ms/step - loss: 1.4704
Epoch 5/20
45584/45584 [==============================] - 91s 2ms/step - loss: 1.4341
Epoch 6/20
45584/45584 [==============================] - 90s 2ms/step - loss: 1.4787
Epoch 7/20
  512/45584 [..............................] - ETA: 1:33 - loss: 1.3172

KeyboardInterrupt: ignored

In [25]:
#generate text
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [29]:
print(generate_text("Julius", 20, model, max_sequence_len))

Julius Milk With Her Sword And Lance She With A Cat With The Bastard Of Orleans Alencon And Reignier Half Ready


In [30]:
print(generate_text("Mercutio", 20, model, max_sequence_len))

Mercutio Henry Is Able To Enrich His Queen And Be A Chair Enter The French And Within The Lieutenant Speaks Within


In [33]:
print(generate_text("Caesar", 20, model, max_sequence_len))

Caesar And Crown The Death Of The Land Beside I I Be A Within On Them And C It Thunders And
